In [1]:
setwd('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1012. NC Well Arsenic/P1012.2. Analyses/P1012.2.2. Arsenic Prediction')
Output = ('/Users/alexis/Library/CloudStorage/OneDrive-UniversityofNorthCarolinaatChapelHill/CEMALB_DataAnalysisPM/Projects/P1012. NC Well Arsenic/P1012.2. Analyses/P1012.2.2. Arsenic Prediction/Output')
cur_date = "052023"

library(readxl)
library(lubridate)
library(tidyverse)
library(gtsummary)
library(caret)
library(e1071)
library(Hmisc)
library(randomForest)
library(pROC)
#library(DMwR) this package only works for oversampling numerical data
library(themis)
library(xgboost)

# reading in files
well_data = data.frame(read_excel("Input/Imputed_Well_Data_051023.xlsx")) 


Attaching package: ‘lubridate’


The following objects are masked from ‘package:base’:

    date, intersect, setdiff, union


── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0     ✔ purrr   1.0.1
✔ tibble  3.2.1     ✔ dplyr   1.1.2
✔ tidyr   1.2.1     ✔ stringr 1.4.1
✔ readr   2.1.2     ✔ forcats 0.5.2
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ lubridate::as.difftime() masks base::as.difftime()
✖ lubridate::date()        masks base::date()
✖ dplyr::filter()          masks stats::filter()
✖ lubridate::intersect()   masks base::intersect()
✖ dplyr::lag()             masks stats::lag()
✖ lubridate::setdiff()     masks base::setdiff()
✖ lubridate::union()       masks base::union()
#BlackLivesMatter

Loading required package: lattice


Attaching package: ‘caret’


The following object is masked from ‘package:purrr’:

    lift


Loading required package: survival


Attaching package: ‘survival’


The foll

In [2]:
head(well_data)

,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Metal,Concentration,Detect_Concentration
,<chr>,<dbl>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<chr>
1,1006004,960,12-155,9/24/12,52,165,41,60.0,7.7,Ar,1.438029,ND
2,1024009,1803,15-177,12/17/15,40,445,42,2.0,7.3,Ar,1.863054,ND
3,1054019,1647,14-212,2/2/15,45,160,40,40.0,7.4,Ar,1.434020,ND
4,1057017,970,12-169,10/22/12,42,440,57,1.5,8.0,Ar,35.000000,D
5,1060006,533,10-239,1/3/11,48,120,42,25.0,7.1,Ar,1.470804,ND
6,1066006,1795,15-224,12/15/15,60,280,32,10.0,8.2,Ar,53.000000,D


Using `water_sample_date`, `casing_depth`, `well_depth`, `static_water_depth`, `pH`, and `flow_rate` to predict As concentration. RF models will be built to predict concentration that has binarized in 3 different ways:

1. Detect vs. Non-Detect
2. < or > 5ppb
3. < or > 10ppb

Starting by creating 2 additional variables for above and below 5 and 10ppb and calculating some summary statistics to determine if there are any signficiant differenes between the predictor variables for each outcome variable.

In [3]:
arsenic_data = well_data %>%
    mutate(Five_Threshold = relevel(factor(ifelse(Concentration >= 5, 1, 0)), ref = "0"),
          Ten_Threshold = relevel(factor(ifelse(Concentration >= 10, 1, 0)), ref = "0"),
           # making this col into a factor
          Detect_Concentration = relevel(factor(ifelse(Detect_Concentration == "D", 1, 0)), ref = "0"),
          # converting water sample date from a character to a date type
          Water_Sample_Date = mdy(Water_Sample_Date)) %>%

    # filtering for Arsenic only
    filter(Metal == "Ar")


head(arsenic_data)

Warning message:
“There was 1 warning in `mutate()`.
ℹ In argument: `Water_Sample_Date = mdy(Water_Sample_Date)`.
Caused by warning:
!  9 failed to parse.”


,Tax_ID,Health_Dept_ID,Permit_No,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Metal,Concentration,Detect_Concentration,Five_Threshold,Ten_Threshold
,<chr>,<dbl>,<chr>,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<dbl>,<fct>,<fct>,<fct>
1,1006004,960,12-155,2012-09-24,52,165,41,60.0,7.7,Ar,1.438029,0,0,0
2,1024009,1803,15-177,2015-12-17,40,445,42,2.0,7.3,Ar,1.863054,0,0,0
3,1054019,1647,14-212,2015-02-02,45,160,40,40.0,7.4,Ar,1.434020,0,0,0
4,1057017,970,12-169,2012-10-22,42,440,57,1.5,8.0,Ar,35.000000,1,1,1
5,1060006,533,10-239,2011-01-03,48,120,42,25.0,7.1,Ar,1.470804,0,0,0
6,1066006,1795,15-224,2015-12-15,60,280,32,10.0,8.2,Ar,53.000000,1,1,1


In [4]:
# summary statistics df: detect_concentration
arsenic_data %>%
  tbl_summary(by = Detect_Concentration, missing = "no", 
  include = colnames(arsenic_data[c(4:9,12)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% # adding p value from anova
  as_tibble()

**Characteristic**,**N**,"**0**, N = 731","**1**, N = 292",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
Water_Sample_Date,"1,020",2013-05-12 (942.199556816559),2012-12-08 (983.231689071497),0.019
Casing_Depth,"1,023",73 (33),56 (23),<0.001
Well_Depth,"1,023",306 (148),340 (137),<0.001
Static_Water_Depth,"1,023",35 (14),37 (16),0.066
Flow_Rate,"1,023",25 (34),14 (17),<0.001
pH,"1,023",7.41 (0.55),7.78 (0.39),<0.001


In [5]:
# summary statistics df: five_threshold
arsenic_data %>%
  tbl_summary(by = Five_Threshold, missing = "no", 
  include = colnames(arsenic_data[c(4:9,13)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% # adding p value from anova
  as_tibble()

**Characteristic**,**N**,"**0**, N = 735","**1**, N = 288",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
Water_Sample_Date,"1,020",2013-05-04 (946.639732018247),2012-12-28 (975.861944951949),0.057
Casing_Depth,"1,023",73 (33),56 (23),<0.001
Well_Depth,"1,023",305 (148),341 (137),<0.001
Static_Water_Depth,"1,023",35 (14),37 (16),0.084
Flow_Rate,"1,023",25 (33),14 (17),<0.001
pH,"1,023",7.41 (0.55),7.79 (0.39),<0.001


In [6]:
# summary statistics df: ten_threshold
arsenic_data %>%
  tbl_summary(by = Ten_Threshold, missing = "no", 
  include = colnames(arsenic_data[c(4:9,14)]), 
              statistic = list(all_continuous() ~ "{mean} ({sd})")) %>%
  add_n() %>% 
  add_p(test = list(all_continuous() ~ "aov")) %>% # adding p value from anova
  as_tibble()

**Characteristic**,**N**,"**0**, N = 819","**1**, N = 204",**p-value**
<chr>,<chr>,<chr>,<chr>,<chr>
Water_Sample_Date,"1,020",2013-05-09 (948.708869245644),2012-10-14 (970.291642552518),0.005
Casing_Depth,"1,023",72 (33),53 (20),<0.001
Well_Depth,"1,023",310 (150),339 (127),0.010
Static_Water_Depth,"1,023",35 (14),37 (16),0.041
Flow_Rate,"1,023",24 (32),13 (16),<0.001
pH,"1,023",7.45 (0.55),7.80 (0.39),<0.001


Although the p values are very significant between the classes of each outcome, there is a high level of class imbalance which is likely to affect model performance.

In [7]:
# creating dfs for each outcome
# dropped 3 rows that had missing dates
arsenic_detect_df = drop_na(arsenic_data[,c(4:9,12)])  
arsenic_five_df = drop_na(arsenic_data[,c(4:9,13)]) 
arsenic_ten_df = drop_na(arsenic_data[,c(4:9,14)])

head(arsenic_detect_df)

,Water_Sample_Date,Casing_Depth,Well_Depth,Static_Water_Depth,Flow_Rate,pH,Detect_Concentration
,<date>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<fct>
1,2012-09-24,52,165,41,60.0,7.7,0
2,2015-12-17,40,445,42,2.0,7.3,0
3,2015-02-02,45,160,40,40.0,7.4,0
4,2012-10-22,42,440,57,1.5,8.0,1
5,2011-01-03,48,120,42,25.0,7.1,0
6,2015-12-15,60,280,32,10.0,8.2,1


# Random Forest
- an ensemble learning method operating by constructing a multitude of decision trees at training time, which uses multiple methods to obtain a better predictive performance and includes bagging and random forest
- algorithm uses a bootstrop aggregation, to reduce overfitting the training datset but only a subset of the features are used hence decorrelation of predictors

In [8]:
rf_classification = function(dataset, outcome, pred_outcome){
    # setting for reproducibility
    set.seed(12)
    # splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) # 5 fold CV
    
    ntree_values = c(50, 250, 500) # number of trees 
    p = dim(dataset)[2] - 1 # number of variables in dataset
    mtry_values = c(sqrt(p), p/2, p/3) # number of predictors

    metrics = data.frame()
    variable_importance_df = data.frame()
    roc_objects = c()
    threshold_data = data.frame()
    
    for (i in 1:length(dataset_index)){
        
        data_train = dataset[-dataset_index[[i]],]
        # using SMOTE to balance the class distribution
        balanced_data_train = smotenc(data_train, outcome)
        data_test = dataset[dataset_index[[i]],]

        # will use ntree and mtry values to determine which combination yields the smallest MSE
        reg_rf_pred_tune = list()
        rf_OOB_errors = list()
        rf_error_df = data.frame()
        for (j in 1:length(ntree_values)){
            for (k in 1:length(mtry_values)){
                reg_rf_pred_tune[[k]] = randomForest(as.formula(paste0(outcome, "~.")), data = balanced_data_train, 
                                                     ntree = ntree_values[j], mtry = mtry_values[k])
                rf_OOB_errors[[k]] = data.frame("Tree Number" = ntree_values[j], "Variable Number" = mtry_values[k], 
                                       "OOB_errors" = reg_rf_pred_tune[[k]]$err.rate[ntree_values[j],1])
                rf_error_df = rbind(rf_error_df, rf_OOB_errors[[k]])
            }
        }

        # finding the lowest OOB error using best number of predictors at split and refitting OG tree
        best_oob_errors <- which(rf_error_df$OOB_errors == min(rf_error_df$OOB_errors))

        reg_rf <- randomForest(as.formula(paste0(outcome, "~.")), data = balanced_data_train,
                               ntree = rf_error_df$Tree.Number[min(best_oob_errors)],
                               mtry = rf_error_df$Variable.Number[min(best_oob_errors)])

        # predicting on test set
        data_test[[pred_outcome]] = predict(reg_rf, newdata = data_test, type = "response")
        
        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                     positive = "1")

        # calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))

        
        # calculating values to plot ROC curve later
        roc_obj = roc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))

        # Return max Youden's index, with specificity and sensitivity
        best_thres_data = data.frame(coords(roc_obj, x = "best", best.method = c("youden", "closest.topleft")))
        threshold_data = rbind(threshold_data, best_thres_data)
        
        # extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
        
        # extracting variable importance
        var_importance_values = data.frame(importance(reg_rf)) %>%
            rownames_to_column(var = "Predictor")
        variable_importance_df = rbind(variable_importance_df, var_importance_values)
   
        # adding values to df
        metrics = rbind(metrics, matrix_values)
        
    }
    
    # taking averages/sd 
    metrics = metrics %>%
        summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
    
    variable_importance_df = variable_importance_df %>%
        group_by(Predictor) %>%
        summarise(MeanDecreaseGini = mean(MeanDecreaseGini)) %>%
        # sorting by most important variables
        arrange(-MeanDecreaseGini)
  
    # return training set, matrix, variable importance values, (last) roc object, best threshold data
    return(list(balanced_data_train, metrics, variable_importance_df, roc_obj, threshold_data))

}

In [9]:
# calling fn
rf_values_arsenic_detect = rf_classification(arsenic_detect_df, "Detect_Concentration", 
                                             "pred_Detect_Concentration")
rf_values_arsenic_five = rf_classification(arsenic_five_df, "Five_Threshold", 
                                             "pred_Five_Threshold")
rf_values_arsenic_ten = rf_classification(arsenic_ten_df, "Ten_Threshold", 
                                             "pred_Ten_Threshold")

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting leve

In [10]:
# viewing results
rf_confusion_matrix = data.frame(Model = rep(c("RF Classification"), times = 3), Kernel = NA,
    Outcome = c("Detect_Concentration", "Five_Threshold", "Ten_Threshold"),
            rbind(rf_values_arsenic_detect[[2]], rf_values_arsenic_five[[2]], rf_values_arsenic_ten[[2]]))

rf_confusion_matrix

# viewing most significant features
rf_values_arsenic_detect[[3]]
rf_values_arsenic_five[[3]]
rf_values_arsenic_ten[[3]]

Model,Kernel,Outcome,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<lgl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
RF Classification,NA,Detect_Concentration,0.6889062,0.5618936,0.8159188,0.5508230,0.6889062
RF Classification,NA,Five_Threshold,0.7022446,0.5834846,0.8210046,0.5634249,0.7022446
RF Classification,NA,Ten_Threshold,0.6905009,0.5096341,0.8713676,0.4984649,0.6905009


Predictor,MeanDecreaseGini
<chr>,<dbl>
pH,138.38219
Casing_Depth,115.46990
Water_Sample_Date,94.06633
Flow_Rate,84.73514
Well_Depth,76.10493
Static_Water_Depth,71.78478


Predictor,MeanDecreaseGini
<chr>,<dbl>
pH,135.48405
Casing_Depth,117.54660
Water_Sample_Date,92.58865
Flow_Rate,88.35422
Well_Depth,76.32078
Static_Water_Depth,73.33457


Predictor,MeanDecreaseGini
<chr>,<dbl>
pH,139.24131
Casing_Depth,135.31228
Water_Sample_Date,102.99642
Flow_Rate,95.85462
Static_Water_Depth,90.68086
Well_Depth,85.85747


RF didn't perform as well as I hoped, so let's try another model - SVM. 

# Support Vector Machine (SVM)
- predicts by projecting predictors onto a high dimensional space and uses kernels to make the data more separable (unfortunately makes interpretability of model results more difficult)
- does a better job at handling a large number of predictors since p > n

In [15]:
# model errors are calculated to assess the predictive accuracy of the model
svm_classification = function(dataset, model, outcome, pred_outcome, gamma_values, cost_values, elsilon_values){
  
    # setting seed for reproducibility
    set.seed(12)
    
    # splitting data into training and testing sets
    dataset_index = createFolds(dataset[[outcome]], k = 5) #5 fold CV
    metrics = data.frame()
    for (i in 1:length(dataset_index)){
        data_train = dataset[-dataset_index[[i]],]
        # using SMOTE to balance the class distribution
        balanced_data_train = smotenc(data_train, outcome)
        
        data_test = dataset[dataset_index[[i]],]

        # now pruning parameters (based on the training dataset to prevent overfitting)
        svr_tune <- tune(svm, as.formula(paste0(outcome, "~.")), data = balanced_data_train, kernel = model, 
                       ranges = list(elsilon = elsilon_values, cost = cost_values, gamma = gamma_values)) 

        # choosing best model
        best_svm <- svr_tune$best.model

        # predicting with tuned parameters 
        data_test[[pred_outcome]] <- predict(best_svm, newdata = data_test, type = "response")

        matrix = confusionMatrix(data = data_test[[pred_outcome]], reference = data_test[[outcome]], 
                                 positive = "1")
 
        # calculating AUC
        auc = auc(response = data_test[[outcome]], predictor = factor(data_test[[pred_outcome]], ordered = TRUE))
        # extracting accuracy, sens, spec, PPV to take mean later
        matrix_values = data.frame(t(c(matrix$byClass[11])), t(c(matrix$byClass[1:3])), auc)
   
        # adding values to df
        metrics = rbind(metrics, matrix_values)
  }
  
  # taking averages/sd 
  metrics = metrics %>%
    summarise(`Balanced Accuracy` = mean(Balanced.Accuracy), Sensitivity = mean(Sensitivity), 
              Specificity = mean(Specificity), PPV = mean(Pos.Pred.Value), AUC = mean(auc))
  
  return(svr_tune)
}

In [16]:
# calling fn
# linear
linear_svm_values_arsenic_detect = svm_classification(arsenic_detect_df, "linear", "Detect_Concentration", 
                                             "pred_Detect_Concentration", 0.035, 1:5, seq(0,1,0.2))
# linear_svm_values_arsenic_five = svm_classification(arsenic_five_df, "linear", "Five_Threshold", 
#                                              "pred_Five_Threshold", 0.035, 1:5, seq(0,1,0.2))
# linear_svm_values_arsenic_ten = svm_classification(arsenic_ten_df, "linear", "Ten_Threshold", 
#                                              "pred_Ten_Threshold", 0.035, 1:5, seq(0,1,0.2))

# # radial
# radial_svm_values_arsenic_detect = svm_classification(arsenic_detect_df, "radial", "Detect_Concentration", 
#                                              "pred_Detect_Concentration", 0.035, 1:5, seq(0,1,0.2))
# radial_svm_values_arsenic_five = svm_classification(arsenic_five_df, "radial", "Five_Threshold", 
#                                              "pred_Five_Threshold", 0.035, 1:5, seq(0,1,0.2))
# radial_svm_values_arsenic_ten = svm_classification(arsenic_ten_df, "radial", "Ten_Threshold", 
#                                              "pred_Ten_Threshold", 0.035, 1:5, seq(0,1,0.2))

# # polynomial
# polynomial_svm_values_arsenic_detect = svm_classification(arsenic_detect_df, "polynomial", "Detect_Concentration", 
#                                              "pred_Detect_Concentration", 0.035, 1:5, 0.1)
# polynomial_svm_values_arsenic_five = svm_classification(arsenic_five_df, "polynomial", "Five_Threshold", 
#                                              "pred_Five_Threshold", 0.035, 1:5, 0.1)
# polynomial_svm_values_arsenic_ten = svm_classification(arsenic_ten_df, "polynomial", "Ten_Threshold", 
#                                              "pred_Ten_Threshold", 0.035, 1:5, 0.1)

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases

Setting levels: control = 0, case = 1

Setting direction: controls < cases



In [28]:
linear_svm_values_arsenic_detect$train.ind

$`(-0.163,117]`
   [1] 1128  935  360  501  352  367   69  807 1133  122  627  624  893 1049
  [15]  854 1038  467   49  464  888  568  929  694  858  864  805  318  836
  [29] 1006  569 1030  759  591 1002   67  304  819    3  161  711  818  450
  [43]   58  323  691 1115  906  680  649  692  241  903  468  288  276  442
  [57]  217  407  365  603  810  242  112  679   59 1033 1015  720  882   76
  [71]   39 1152   98 1119 1127  105   77  593  616  631  990  967  279  457
  [85]  899  563  954 1077   20  938   56  159   31  221 1014  268 1048  879
  [99]  440  597 1013  559  545 1058  958  101  334  131 1012  223  756  828
 [113]  332  504  348  216   70  240   11   88 1095  177 1163  727 1160  991
 [127]  158  411 1150  634  270  902 1130  987  381 1022  946  378  966  992
 [141]  601  136  230  118  723  964  928  175  480  688 1131  150 1007 1103
 [155] 1043  681  744  255  247  518  768  553  979  129  664  469 1045  511
 [169]  213  473  424  479 1062  127 1044  106  536  916  617  417  321  722
 [183]  219  394   17  267  978  340  404  717   48  696  660  165  564  931
 [197]  669 1136  167  351  905 1029  852  713  973  113 1018  824  699  448
 [211]   64  922  164  472  708  166  746  455  613   71  508  147  345  581
 [225]   94  152 1067  995  489  885  338   90  576   34  385  933  792  948
 [239]  265  571   52  604  295  366  629  682  203 1153  642  636  666  141
 [253] 1050  718  107  356   30  782  747 1001   22 1148  429  308  942 1104
 [267]  372  510  533  401  565  277  426  392   33  316  520 1086   81 1098
 [281] 1035  923  382 1079  891  793    8  753 1025  412  233  675 1060   38
 [295]  772  750  619  199 1065  488  769  730  579  130  422  921  140  661
 [309]  623  578  125  471  674  985 1092 1144  124  831  474  949  728   37
 [323]  303  456 1154  789 1082  671  815  388  151   62  331  190 1036 1149
 [337] 1084  174  337  980  687  350 1146 1070  232  795  249  901  794  952
 [351]  974  182  148   86 1143 1016 1132  490  500  895  574  612  994  250
 [365]  399  911  260  639  874  333  379  369  846  117  436 1059  393  780
 [379]  909 1120  740  788 1041 1114  466  552 1063 1108  599 1093  344  109
 [393]  325  183   36  925  494  299  172  896  725   44  258  887  215  959
 [407]  821   65  734  224  833  228  570  886  374  204  406 1005   80 1088
 [421]  930  102  575  132  181  355   60   46   61  525  273  856   14 1151
 [435] 1052  220  420  822 1161  222  586  862  298  513   68 1028 1158  743
 [449]  731  968  880  302 1107 1081  709  259  982  144  969  296  580  202
 [463]  194  751  706  844  651    6  335  932  425  264    4  596 1021  398
 [477]  128   74  760  460 1074  762  179  635  870  614  567  767  438  558
 [491]  317  594  881  384 1061  804  719  834  670  548  287  754   19  878
 [505]  535  297  437  419  286  630  961   43  380  650  643  257  602  482
 [519]  848 1057    9  984  790   92 1113   83  495   96  524  806  293  439
 [533] 1112  390 1123  801  301   28  377   35  275  198  280  138  294  890
 [547]  693  271  509  512  748  191  716  873  492  976  205  432 1099 1009
 [561] 1139 1039 1101  889  546 1117 1027  764  244  538  940  796 1064  595
 [575] 1069  685  761   42  402  349 1055  721  110  726  195 1087  120  389
 [589] 1147  757  714  530  506  924 1072  441  395  813  855  516  434  829
 [603] 1110  919  342  364   85 1129  414  403   27  817 1000  529 1010  646
 [617]  484   87  550  851  336  738  972  840  193  913 1116  313  777  989
 [631] 1020 1042  672  915   21  234  231  839  210  820  201  963  871  996
 [645]  799  951  749  633  695  698  461  246  655  433  114  549  644  816
 [659]  977  965  981  507  447 1122  443  770  941  119  657  872  263  481
 [673]  176  587 1091  843  531 1164  993 1145  115  163 1056    2  957  192
 [687]  798   79  863   57  632  809  841  758 1140  825  830  252  502  354
 [701]  773  315   99  752  357  274  707  584  849  449  196 1085  936 1054
 [715]  566  142  653  791  514    7  475  519  423  850 

In [13]:
# viewing results
svm_confusion_matrix = data.frame(Model = "SVM Classification",
        Outcome = c("Detect_Concentration", "Five_Threshold", "Ten_Threshold"),
        Kernel = rep(c(rep(c("linear"), times = 3), rep(c("radial"), times = 3), rep(c("polynomial"), times = 3))),
            rbind(linear_svm_values_arsenic_detect, linear_svm_values_arsenic_five, linear_svm_values_arsenic_ten, 
            radial_svm_values_arsenic_detect, radial_svm_values_arsenic_five, radial_svm_values_arsenic_ten, 
            polynomial_svm_values_arsenic_detect, polynomial_svm_values_arsenic_five, 
            polynomial_svm_values_arsenic_ten))

svm_confusion_matrix

Model,Outcome,Kernel,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SVM Classification,Detect_Concentration,linear,0.7202201,0.7948568,0.6455834,0.4733987,0.7202201
SVM Classification,Five_Threshold,linear,0.7203431,0.7849970,0.6556891,0.4731675,0.7203431
SVM Classification,Ten_Threshold,linear,0.7285560,0.8136585,0.6434535,0.3648138,0.7285560
SVM Classification,Detect_Concentration,radial,0.7085780,0.7742840,0.6428720,0.4648690,0.7085780
SVM Classification,Five_Threshold,radial,0.7158280,0.7814277,0.6502283,0.4683337,0.7158280
SVM Classification,Ten_Threshold,radial,0.7286260,0.7647561,0.6924959,0.3862495,0.7286260
SVM Classification,Detect_Concentration,polynomial,0.6585424,0.9283460,0.3887388,0.3781675,0.6585424
SVM Classification,Five_Threshold,polynomial,0.6552732,0.9131276,0.3974187,0.3746065,0.6552732
SVM Classification,Ten_Threshold,polynomial,0.6538684,0.9069512,0.4007856,0.2753551,0.6538684


In [14]:
# combining in 1 df
final_df = full_join(rf_confusion_matrix, svm_confusion_matrix) %>%
    arrange(Outcome, -Balanced.Accuracy)

final_df

Joining with `by = join_by(Model, Kernel, Outcome, Balanced.Accuracy,
Sensitivity, Specificity, PPV, AUC)`


Model,Kernel,Outcome,Balanced.Accuracy,Sensitivity,Specificity,PPV,AUC
<chr>,<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
SVM Classification,linear,Detect_Concentration,0.7202201,0.7948568,0.6455834,0.4733987,0.7202201
SVM Classification,radial,Detect_Concentration,0.7085780,0.7742840,0.6428720,0.4648690,0.7085780
RF Classification,NA,Detect_Concentration,0.6889062,0.5618936,0.8159188,0.5508230,0.6889062
SVM Classification,polynomial,Detect_Concentration,0.6585424,0.9283460,0.3887388,0.3781675,0.6585424
SVM Classification,linear,Five_Threshold,0.7203431,0.7849970,0.6556891,0.4731675,0.7203431
SVM Classification,radial,Five_Threshold,0.7158280,0.7814277,0.6502283,0.4683337,0.7158280
RF Classification,NA,Five_Threshold,0.7022446,0.5834846,0.8210046,0.5634249,0.7022446
SVM Classification,polynomial,Five_Threshold,0.6552732,0.9131276,0.3974187,0.3746065,0.6552732
SVM Classification,radial,Ten_Threshold,0.7286260,0.7647561,0.6924959,0.3862495,0.7286260
